In [1]:
import cv2

In [2]:
def generate_dataset():
    face_classifier = cv2.CascadeClassifier("haarcascade_frontalface_default.xml")
    def face_cropped(img):
        gray = cv2.cvtColor(img,cv2.COLOR_BGR2GRAY)
        faces = face_classifier.detectMultiScale(gray, 1.3, 5)

        if faces is ():
            return None
        for (x,y,w,h) in faces:
            cropped_face = img[y:y+h,x:x+w]
        return cropped_face
    
    cap = cv2.VideoCapture(1)
    id =4
    img_id = 0
     
    while True:
        ret, frame = cap.read()
        if face_cropped(frame) is not None:
            img_id+=1
            face = cv2.resize(face_cropped(frame), (200,200))
            face = cv2.cvtColor(face, cv2.COLOR_BGR2GRAY)
            file_name_path = "data/user."+str(id)+"."+str(img_id)+".jpg"
            cv2.imwrite(file_name_path, face)
            cv2.putText(face, str(img_id), (50,50), cv2.FONT_HERSHEY_COMPLEX, 1, (0,255,0), 2)
             
            cv2.imshow("Cropped face", face)
             
        if cv2.waitKey(1)==13 or int(img_id)==200: #13 is the ASCII character of Enter
            break
             
    cap.release()
    cv2.destroyAllWindows()
    print("Collecting samples is completed....")
#generate_dataset()

<>:7: SyntaxWarning: "is" with 'tuple' literal. Did you mean "=="?
<>:7: SyntaxWarning: "is" with 'tuple' literal. Did you mean "=="?
C:\Users\user\AppData\Local\Temp\ipykernel_6276\156126693.py:7: SyntaxWarning: "is" with 'tuple' literal. Did you mean "=="?
  if faces is ():


In [3]:
import os
import numpy as np
from PIL import Image
import cv2

In [4]:
def train_classifier(data_dir):
    path = [os.path.join(data_dir, f) for f in os.listdir(data_dir)]

    faces = []
    ids = []

    for image in path:
        img = Image.open(image).convert('L')
        imageNp = np.array(img, 'uint8')
        id = int(os.path.split(image)[1].split(".")[1])

        faces.append(imageNp)
        ids.append(id)

    ids = np.array(ids)

    # Train and save classifier
    clf = cv2.face.LBPHFaceRecognizer_create()
    clf.train(faces, ids)
    clf.write("classifier.xml")


train_classifier("data")

In [1]:
import cv2
import numpy as np
from PIL import Image
import os

In [2]:
import cv2
import requests
import numpy as np
from PIL import Image
import os
import mysql.connector
from mysql.connector import Error

# 데이터베이스에 연결하는 함수
def create_connection():
    connection = None
    try:
        connection = mysql.connector.connect(
            host='localhost',
            user='root',
            password='1234',
            database='Project_MDP'
        )
        if connection.is_connected():
            print("Successfully connected to the database")  # 데이터베이스 연결 성공 시 출력
    except Error as e:
        print(f"Error: '{e}'")  # 연결 실패 시 오류 메시지 출력
    
    return connection

# 데이터베이스에서 face_id를 확인하는 함수
def check_id_in_database(face_id):
    connection = create_connection()
    cursor = connection.cursor()
    query = "SELECT COUNT(*) FROM results WHERE face_id = %s"
    cursor.execute(query, (face_id,))
    result = cursor.fetchone()[0]
    cursor.close()
    connection.close()
    return result > 0  # face_id가 데이터베이스에 있으면 True 반환

# 얼굴 인식을 위한 경계와 텍스트를 그리는 함수
def draw_boundary(img, classifier, scaleFactor, minNeighbors, color, text, clf, recognized_ids):
    gray_img = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)  # 이미지를 회색조로 변환
    features = classifier.detectMultiScale(gray_img, scaleFactor, minNeighbors)  # 얼굴 특징 검출
    face_id_motor = None  # 인식된 face_id를 저장할 변수 초기화
     
    for (x, y, w, h) in features:
        cv2.rectangle(img, (x, y), (x+w, y+h), color, 2)  # 얼굴 주위에 사각형 그리기
         
        id, pred = clf.predict(gray_img[y:y+h, x:x+w])  # 얼굴 인식 예측
        confidence = int(100 * (1 - pred / 300))  # 신뢰도 계산
         
        if confidence > 70:  # 신뢰도가 70% 이상일 때
            face_id = None
            if id == 1:
                face_id = "KHB"
            elif id == 2:
                face_id = "KSS"
            elif id == 3:
                face_id = "CHW"
            elif id == 4:
                face_id = "JMW"
            elif id == 5:
                face_id = "KHS"
            
            if face_id and face_id not in recognized_ids:
                cv2.putText(img, face_id, (x, y-5), cv2.FONT_HERSHEY_SIMPLEX, 0.8, color, 1, cv2.LINE_AA)  # 얼굴 ID 텍스트 출력
                if check_id_in_database(face_id):
                    print(f"Recognized face_id: {face_id}")  # 인식된 얼굴 ID 출력
                    face_id_motor = id  # face_id_motor에 ID 저장
                recognized_ids.add(face_id)  # 인식된 ID를 집합에 추가
        else:
            cv2.putText(img, "UNKNOWN", (x, y-5), cv2.FONT_HERSHEY_SIMPLEX, 0.8, (0, 0, 255), 1, cv2.LINE_AA)  # 알 수 없는 얼굴 텍스트 출력
    return img, face_id_motor  # 처리된 이미지와 face_id_motor 반환

faceCascade = cv2.CascadeClassifier("haarcascade_frontalface_default.xml")  # 얼굴 검출을 위한 Haar Cascade 로드

clf = cv2.face.LBPHFaceRecognizer_create()  # 얼굴 인식기를 위한 LBPHFaceRecognizer 생성
clf.read("classifier.xml")  # 사전 학습된 분류기 로드

video_capture = cv2.VideoCapture(0)  # 웹캠 비디오 캡처 시작

recognized_ids = set()  # 인식된 얼굴 ID를 저장할 집합 초기화

while True:
    ret, img = video_capture.read()  # 프레임 읽기
    img, face_id_motor = draw_boundary(img, faceCascade, 1.3, 6, (255, 255, 255), "Face", clf, recognized_ids)  # 얼굴 검출 및 인식 수행
    cv2.imshow("Face Detection", img)  # 결과 이미지 표시
    
    if face_id_motor:
        print(f"face_id_motor = {face_id_motor}")  # face_id_motor가 설정된 경우 출력
     
    if cv2.waitKey(1) == 13:  # Enter 키를 누르면 루프 종료
        break

video_capture.release()  # 비디오 캡처 해제
cv2.destroyAllWindows()  # 모든 OpenCV 창 닫기

Successfully connected to the database
Recognized face_id: KHB
face_id_motor = 1
Successfully connected to the database
Recognized face_id: KHS
face_id_motor = 5
Successfully connected to the database
Recognized face_id: KSS
face_id_motor = 2
